In [11]:
%pylab --no-import-all
%matplotlib inline
import seaborn as sns
import sys
import pandas as pd
from ipywidgets import widgets
from IPython.display import Markdown, display, clear_output, Audio
import os
from os.path import join, isdir
from glob import iglob
import json
from pprint import pformat
import utils
import pickle
import io
import base64
from scipy.io import wavfile
import birdsonganalysis as bsa

sys.path.append('../model')

from song_model import SongModel

sns.set_palette('colorblind')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [12]:
gridpath = '../model/res/nb_pad_gridsearch_170411_140018/'
run_paths = sorted([run_path for run_path in iglob(join(gridpath, '*')) if isdir(run_path)])

gridanalysis = utils.GridAnalyser(run_paths)

In [13]:
with open(join(gridpath, 'desc.md')) as f:
    display(Markdown(f.read()))

# Run description

## Goals

- See if padding problem can be corrected with larger padding

## What have changed compared to previous runs?

- Padding size can be increased.

## Expected Results (Success condition)

- With a certain padding width, there should no longer be padding learning issues

<!-- ## Notes -->


In [14]:
vbox = widgets.VBox()

tb1 = widgets.ToggleButtons()
tb1.options = ['on_alpha_silence_beta', 'zero_alpha_zero_beta',
              'on_alpha_zero_beta', 'oscilating_alpha_sound_oscilating_beta',
              'zero_alpha_silence_beta', 'zero_alpha_sound_beta']
display(tb1)
tb2 = widgets.ToggleButtons()
tb2.options = ['amp_entropy_only', 'default_coef', 'amp_pitch_entropy_only']
display(tb2)
tb3 = widgets.ToggleButtons()
tb3.options = ['big_ba', 'big_simple']
display(tb3)
tb4 = widgets.ToggleButtons()
tb4.options = ['fast_dev', 'slow_dev']
display(tb4)

def find_id(value):
    for i in range(len(gridanalysis.run_paths)):
        if gridanalysis.conf[i]['name'] == '_'.join([tb1.value,tb2.value, tb3.value, tb4.value]):
            slider.value = i
            break

slider = widgets.IntSlider(min=0, max=len(run_paths)-1)
slider.observe(lambda x: gridanalysis.show(x['new'], vbox), names='value')
display(slider)
display(vbox)

tb1.observe(lambda x: find_id(x['new']), names='value')
tb2.observe(lambda x: find_id(x['new']), names='value')
tb3.observe(lambda x: find_id(x['new']), names='value')
tb4.observe(lambda x: find_id(x['new']), names='value')


In [15]:
display(slider)